In [1]:
import os
from dotenv import dotenv_values
from tqdm.notebook import tqdm
import numpy as np

env = dotenv_values()
txtFilemame = "enwiki-20220601-pages-articles-multistream.xml"
txtPath = os.path.join(env['STORE_PATH'],txtFilemame)
n = 4
histOutputPath = os.path.join(env['STORE_PATH'],"hist89-{}gram.mm".format(n))
print(f"{txtPath=}")
print(f"{histOutputPath=}")

txtPath='/mnt/nvme/btrfs/enwiki-20220601-pages-articles-multistream.xml'
histOutputPath='/mnt/nvme/btrfs/hist89-4gram.mm'


In [2]:
%%time
blocksize = 0xC0000000 * 4 #bytes
fp = np.memmap(histOutputPath, dtype=np.uint32, mode='w+', shape=(4294967296,))
txt = np.memmap(txtPath,mode='r',dtype=np.uint8)
def view(txt,i,offset):
    start = blocksize*i+offset
    end = blocksize*(i+1)+offset
    view = txt[start:end]
    view = txt[start:(start+len(view)-len(view)%4)]
    return view.view(np.uint32)
create_bincount = lambda i,offset : np.bincount(view(txt,i,offset),minlength=0x100000000)
for i, offset in tqdm(np.ndindex((int(len(txt)/blocksize)+1,4))):
    print(f"{i=}, {offset=}")
    %time fp+=create_bincount(i,offset).astype(dtype=np.uint32,copy=False)
!chmod 444 "$histOutputPath"

0it [00:00, ?it/s]

i=0, offset=0


In [ ]:
n = 4
fp = np.memmap('/mnt/nvme/btrfs/hist89-{}gram.mm'.format(n), dtype=np.uint32, mode='r', shape=(4294967296,))